# 2 EDA: Breast Cancer Gene Expressions

## 2.1 Contents <a id='2.1_Contents'></a>

* [2 Exploratory Data Analysis](#2_EDA)
    * [2.1 Contents](#2.1_Contents)
    * [2.2 Introduction](#2.2_Introduction)
    * [2.3 Imports](#2.3_Imports)
    * [2.4 Loading the Data](#2.4_Loading)
    * [2.5 Clinical Data](#2.5_Clinical)
        *  [2.5.1 Feature Engineering Outcomes](#2.5.1_Feature)
        *  [2.5.2 Distribution of Clinical Attributes](#2.5.2_Distribution_Clinical)
        *  [2.5.3 Clinical Attributes vs. Outcomes](#'2.5.3_Attributes_Outcomes)
        *  [2.5.4 Are there similar distributions in the data for the different cohorts?](#2.5.4_Cohorts)
        *  [2.5.5 Correlations Between Clinical Attributes](#2.5.5_Attributes_Correlations)
        *  [2.5.5a Impute the Missing Tumor Size](#2.5.5a_impute_tumor_size)
        *  [2.5.6 Correlation Matrix](#2.5.6_correlation_matrix)
    * [2.6 Z Score Data](#2.6_z_score)
        *  [2.6.1 How are the z scores connected to the genetic mutation data?](#2.6.1_z_score_mutation)
        *  [2.6.2 Z Score Outcomes](#2.6.2_z_score_outcomes)
        *  [2.6.2a Z Score Distributions](#2.6.2a_z_score_dist)
        *  [2.6.2b P-Values for Z Scores](#2.6.2a_p_values)
        *  [2.6.3 Correlation of Z Scores and Outcome](#2.6.3_corr)
    *  [2.7 Genetic Mutation Data](#2.6.7_genetic)
        *  [2.7.1 Convert gene mutation columns into binary](#2.7.1_binary)
        *  [2.7.2 Correlations of Mutation Data](#2.7.2_corr)
    *  [2.8 Preparation for Export](#2.8_prep)
    *  [2.9 Explort the Data](#2.9_export)
    *  [2.10 Summary](#2.10_summary)
    

## 2.2 Introduction <a id='2.2_Introduction'></a>

In the previous notebook, I cleaned and wrangled the Metabric Breast Cancer Gene Expression Profiles data, and now I will do exploratory data analysis. The data was sourced from https://www.kaggle.com/datasets/raghadalharbi/breast-cancer-gene-expression-profiles-metabric. 

In this notebook I will explore these questions: 

Questions:
* What is the distribution of the different clinical attributes?
* How are the z scores connected to the genetic mutation data? (i.e. are there z scores associated with each mutation?)
* Can we drop the 'Stage' data entirely based on other clinical attributes that have less missing data? That is the one missing so much. 
* Are there similar distributions in the data for the different cohorts? 
* What does it look like when we compare presence of genetic mutations with survival? 


NOTE the z scores correspond to a gene in the gene mutations. Need to look at this and somehow link then, as the z scores show how much those genes are up or downregulated. 

should the patients be grouped by similar attributes? would we want to do unsupervised learning to cluster them and then take a classification approach? the premise of the business problem is that patients with the same clinical attributes have different outcomes, so should we take this into account?

also see where there are many more z scores than gene mutations - how do they match up? Why aren't there the same number? Is this EDA or wrangling?


## 2.3 Imports  <a id='2.3_Imports'></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pyarrow.parquet as pq
from scipy.stats import pearsonr
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
import statsmodels.api as sm
from scipy import stats
import missingno as msno
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

In [ ]:
# setting the display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## 2.4 Loading the Data <a id='2.4_Loading'></a>

In [ ]:
# importing using parquet to preserve data types

cancer_data=pd.read_parquet(r'C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\capstone 2\cancer_data_cleaned.parquet', engine='pyarrow')

In [ ]:
cancer_data.info()

checking my data types. Here I can see the mix of types for our clinical data, that the z scores are floats, and the mutations are categories. 

In [ ]:
data_types= cancer_data.dtypes
data_types.head(32)

In [ ]:
data_types.tail()

In [ ]:
cancer_data.shape

In [ ]:
cancer_data.describe()

## 2.5  Clinical Data <a id='2.5_Clinical'></a>

The clinical attributes are the section of the data where it should be the easiest to see patterns from simple graphical representation of the data without manipulation. Here I will explore the clinical attributes and see what stands out. 

### 2.5.1 Feature Engineering Outcomes <a id='2.5.1_Feature'></a>

In [ ]:
#looking at survival rates - 0 = died, 1 = lived.

survival_counts = cancer_data['overall_survival'].value_counts()
print('Percents Died vs Survived:\n',100*survival_counts/len(cancer_data))

How does this compare to the more detailed breakdown of outcomes in the death_from_cancer column?

In [ ]:
cancer_survival_counts = cancer_data['death_from_cancer'].value_counts()
print('Outcomes:\n',100*cancer_survival_counts/len(cancer_data))

In [ ]:
ax = sns.countplot(x='death_from_cancer', data=cancer_data)
plt.xlabel('Outcome')
plt.ylabel('Count')
plt.title('Survival Outcomes')
plt.show()

Here we can see that about 33% of patients in this dataset died from breast cancer, 25% died of other causes, and 42% were alive a the end of the study. For our purposes, we only should look at this metric, as it is more detailed.

This is an unbalanced classification problem, where patients who lived are the majority class.

As we are interested in death due to breast cancer, we can combine the other outcomes (lived/death from other causes) together. 

In [ ]:
# first change type from category to object:
cancer_data['death_from_cancer'] = cancer_data['death_from_cancer'].astype('str')

# Change to binary, where 0 means died from cancer
cancer_data.loc[cancer_data['death_from_cancer'] != 'Died of Disease', 'death_from_cancer'] = 1
cancer_data.loc[cancer_data['death_from_cancer'] == 'Died of Disease', 'death_from_cancer'] = 0

# change data type back to category
cancer_data['death_from_cancer'] = cancer_data['death_from_cancer'].astype('category')

In [ ]:
cancer_data['death_from_cancer'].head()

In [ ]:
#show how much of our data died of cancer (0) and did not die of cancer (1).
death_counts = cancer_data['death_from_cancer'].value_counts()

print('Counts for Did Not Die of Disease(1) vs Percents Died of Disease(0)\n',death_counts)

print('\n Did Not Die of Disease(1) vs Percents Died of Disease(0):\n',100*death_counts/len(cancer_data))

In [ ]:
# plotting our newly reformatted death_from_cancer
ax = sns.countplot(x='death_from_cancer', data=cancer_data)
plt.xlabel('Outcome (0:Died of Disease, 1:Survived/Death from other causes)')
plt.ylabel('Count')
plt.title('Survival Outcomes')
plt.show()

### 2.5.2 Distribution of Clinical Attributes <a id='2.5.2_Distribution_Clinical'></a>

Below I will look at plots of my clinical data. First, I'm looking at value counts for categorical clinical data. I'm splitting them up into multiple blocks so that they are more readable.

In [ ]:
# doing sub plots in 3x2 grids
fig, ax = plt.subplots(2,3, figsize=(12,8))
cancer_data.overall_survival.value_counts().plot(kind='bar', ax=ax[0,0])
ax[0,0].set_title('Overall Survival')
ax[0,0].set_xlabel('Survival')

cancer_data.type_of_breast_surgery.value_counts().plot(kind='bar', ax=ax[0,1])
ax[0,1].set_title('Type of Breast Surgery')

cancer_data.chemotherapy.value_counts().plot(kind='bar', ax=ax[0,2])
ax[0,2].set_title('Chemotherapy ')

cancer_data.age_at_diagnosis.plot(kind='hist', ax=ax[1,0])
ax[1,0].set_title('Age at Diagnosis ')
ax[1,0].set_xlabel('Age')


cancer_data.cellularity.value_counts().plot(kind='bar', ax=ax[1,1])
ax[1,1].set_title('Cellularity')

cancer_data.cancer_type_detailed.value_counts().plot(kind='bar', ax=ax[1,2])
ax[1,2].set_title('Detailed Cancer Type')

plt.subplots_adjust(wspace=0.5, hspace=1.2);


In [ ]:

fig, ax = plt.subplots(2,3, figsize=(12,8))
cancer_data['pam50_+_claudin-low_subtype'].value_counts().plot(kind='bar', ax=ax[0,0])
ax[0,0].set_title('pam50_+_claudin-low_subtype')

cancer_data.er_status_measured_by_ihc.value_counts().plot(kind='bar', ax=ax[0,1])
ax[0,1].set_title('er_status_measured_by_ihc')

cancer_data.neoplasm_histologic_grade.value_counts().plot(kind='bar', ax=ax[0,2])
ax[0,2].set_title('neoplasm_histologic_grade')

cancer_data.her2_status.value_counts().plot(kind='bar', ax=ax[1,0])
ax[1,0].set_title('her2_status  ')

cancer_data.tumor_other_histologic_subtype  .value_counts().plot(kind='bar', ax=ax[1,1])
ax[1,1].set_title('tumor_other_histologic_subtype')

cancer_data.hormone_therapy  .value_counts().plot(kind='bar', ax=ax[1,2])
ax[1,2].set_title('hormone_therapy  ')

plt.subplots_adjust(wspace=0.5, hspace=.7);

In [ ]:

fig, ax = plt.subplots(2,3, figsize=(12,8))
cancer_data['inferred_menopausal_state'].value_counts().plot(kind='bar', ax=ax[0,0])
ax[0,0].set_title('inferred_menopausal_state')

cancer_data.integrative_cluster.value_counts().plot(kind='bar', ax=ax[0,1])
ax[0,1].set_title('integrative_cluster ')

cancer_data.primary_tumor_laterality.value_counts().plot(kind='bar', ax=ax[0,2])
ax[0,2].set_title('primary_tumor_laterality')

cancer_data.lymph_nodes_examined_positive.value_counts().plot(kind='hist', bins=50, ax=ax[1,0]) 
ax[1,0].set_title('lymph_nodes_examined_positive')

cancer_data.mutation_count.plot(kind='hist',bins=50, ax=ax[1,1])
ax[1,1].set_title('mutation count')
ax[1,1].set_xlim([0, 30])

cancer_data.overall_survival_months.plot(kind='hist', ax=ax[1,2])
ax[1,2].set_title('overall_survival_months')

plt.subplots_adjust(wspace=0.5, hspace=.5);

In [ ]:

fig, ax = plt.subplots(4, figsize=(8, 20))

cancer_data['pr_status'].value_counts().plot(kind='bar', ax=ax[0])
ax[0].set_title('pr_status  ')

cancer_data.radio_therapy.value_counts().plot(kind='bar', ax=ax[1])
ax[1].set_title('radio_therapy')

cancer_data['3-gene_classifier_subtype'].value_counts().plot(kind='bar', ax=ax[3])
ax[3].set_title('3-gene_classifier_subtype')

cancer_data.tumor_size.value_counts().plot(kind='hist',bins=50, ax=ax[2])
ax[2].set_title('tumor size (cm)')
ax[2].set_xlim(0, 80)

plt.subplots_adjust(wspace=0.5, hspace=.5)
plt.show();

In [ ]:
# find the average age of patients in this study

average_age = cancer_data.age_at_diagnosis.mean()
rounded_average_age = round(average_age, 2)
print('average age is ', rounded_average_age)

median_age = cancer_data.age_at_diagnosis.median()
rounded_median_age = round(median_age, 2)
print('median age is ', rounded_median_age)

Our median and average age are both between 61 and 62. 

### Observations:

The proportions for death and getting a mastectomy are similar - I'm wondering if the patients that had breast conserving surgery had less severe disease and therefore were more likely to survive. I will look at this below in more detail.

Cancer type - the vast majority of cases were invasive ductal carcinoma

Age follows a rougly normal distribution, peaking around age 70. Most women were post menopausal, which makes sense given the age distribution.

Most patients did not get chemo, but did get radiotherapy and hormone therapy. The proportions with death and getting radiotherapy are roughly the same, and again I wonder if this is because more severe disease was present.

HER2 status for most patients was negative, ER status mostly positive

Most neoplasms were grade 3

Mutation counts peaked at 7

Positive lymph nodes looks to fall roughly exponentially

Left and right sides are about equal - I think we can take this out because I dont think it is relavent to survival

Tumor stage and tumor size have similar distributions (both left skewed), and I'm guessing they are usually similar

PR status roughly equal, but a little more positive.

### 2.5.3 Clinical Attributes vs. Outcomes  <a id='2.5.3_Attributes_Outcomes'></a>

### Treatment modality survival comparison

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,8))
cancer_data.groupby('chemotherapy')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0, 0])
ax[0, 0].set_title('Chemotherapy vs. Cancer Survival')

cancer_data.groupby('type_of_breast_surgery')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0, 1])
ax[0, 1].set_title('Type of Surgery vs. Cancer Survival')

cancer_data.groupby('hormone_therapy')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1,0])
ax[1, 0].set_title('Type of Surgery vs. Cancer Survival')

cancer_data.groupby('radio_therapy')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1,1])
ax[1, 1].set_title('Radiotherapy vs. Cancer Survival')

#Remove legends in subplots:
for a in ax.flat:
    if a.get_legend():
        a.get_legend().remove()

# common legend outside the subplots
handles, labels = ax[0, 0].get_legend_handles_labels()
fig.legend(handles,  ['Died of Disease', 'Survived or Died of Other Causes'], loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2)


plt.subplots_adjust(wspace=0.5, hspace=1.3)
plt.show();


Hormone Therapy and Radiotherapy have roughly the same outcome for those who died of concer and those who did not. It looks like more people died of cancer who did not get chemotherapy and who got breast conserving surgery. 

### Tumor Size vs. Outcome

In [ ]:
fig, ax = plt.subplots(2, figsize=(12,8))

cancer_data[cancer_data['death_from_cancer'] == 0]['tumor_size'].plot.hist(ax=ax[0], alpha=0.5, stacked=True, range=[0,100], bins=30)
cancer_data[cancer_data['death_from_cancer'] == 1]['tumor_size'].plot.hist(ax=ax[0], alpha=0.5, stacked=True,range=[0,100], bins=30)
ax[0].set_title('Tumor Size vs. Outcome')
ax[0].set_xlabel('Tumor Size')

cancer_data.groupby('tumor_stage')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1])
ax[1].set_title('tumor_stage vs. Outcome')

plt.subplots_adjust(wspace=0.5, hspace=.5)
plt.show();


The tumor stage has a clear relationship with outcomes. This is also unfortunately the feature that one of the cohorts didn't record data for, so we are missing about 26% of the data from this column. 

### Plots of Clinical Attributes vs Outcomes

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(12,8))

#doing a histogram for age data
cancer_data[cancer_data['death_from_cancer'] == 0]['age_at_diagnosis'].plot.hist(ax=ax[0, 0], alpha=0.7, color='blue', stacked=True)
cancer_data[cancer_data['death_from_cancer'] == 1]['age_at_diagnosis'].plot.hist(ax=ax[0, 0], alpha=0.5, color='orange', stacked=True)
ax[0, 0].set_title('Age at Diagnosis vs. Outcomes')
ax[0,0].legend(['Died of disease', 'Did not die of disease'])


cancer_data.groupby('cancer_type_detailed')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0,1])
ax[0, 1].set_title('Cancer Type vs. Outcomes')
ax[0, 1].set_xticklabels(['Breast', 'Inv. Ductal', 'Inv. Lob.','Inv. Mixed Muc.', 'Mixed Duct/Lob', 'Metaplastic'])

cancer_data.groupby('tumor_other_histologic_subtype')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0,2])
ax[0, 2].set_title('Tumor Subtype vs. Outcomes')

cancer_data.groupby('neoplasm_histologic_grade')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1,0])
ax[1, 0].set_title('Neoplasm Histologic Grade vs. Outcomes')

cancer_data.groupby('primary_tumor_laterality')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1,1])
ax[1,1].set_title('Primary Tumor Laterality vs. Outcomes')

cancer_data[cancer_data['death_from_cancer'] == 0]['lymph_nodes_examined_positive'].plot.hist(ax=ax[1,2], alpha=0.5, color = 'blue', stacked=True,range=[0,15], bins=30)
cancer_data[cancer_data['death_from_cancer'] == 1]['lymph_nodes_examined_positive'].plot.hist(ax=ax[1,2], alpha=0.5, color = 'orange', stacked=True, range=[0,15],bins=30)
ax[1, 2].set_title('Positive Lymph Nodes vs. Outcomes')
ax[1,2].set_xlabel('Number of Positive Lymph Nodes')

cancer_data[cancer_data['death_from_cancer'] == 0]['mutation_count'].plot.hist(ax=ax[2,0], alpha=0.5, color = 'blue', stacked=True, range=[0,20], bins=30)
cancer_data[cancer_data['death_from_cancer'] == 1]['mutation_count'].plot.hist(ax=ax[2,0], alpha=0.5, color = 'orange', stacked=True, range=[0,20],bins=30)
ax[2,0].set_title('Mutation Count vs. Outcomes')


cancer_data.groupby('inferred_menopausal_state')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[2,1])
ax[2, 1].set_title('Menopausal State vs. Outcomes')


cancer_data.groupby('cellularity')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[2, 2])
ax[2, 2].set_title('Cellularity vs. Outcomes')


#Remove legends in subplots:
for a in ax.flat:
    if a.get_legend():
        a.get_legend().remove()

# common legend outside the subplots
handles, labels = ax[2, 1].get_legend_handles_labels()
fig.legend(handles,  ['Died of Disease', 'Survived or Died of Other Causes'], loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2)


plt.subplots_adjust(wspace=0.4, hspace=2.2)

plt.show();

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12,8))
cancer_data.groupby('pam50_+_claudin-low_subtype')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0, 0])
ax[0, 0].set_title('Pam50 Claudin Low Subtype vs. Outcomes')

cancer_data.groupby('er_status_measured_by_ihc')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0, 1])
ax[0,1].set_title('ER Status vs. Outcomes')

cancer_data.groupby('her2_status')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0,2])
ax[0, 2].set_title('HER2 Status vs. Outcomes')


cancer_data.groupby('integrative_cluster')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1, 0])
ax[1, 0].set_title('Integrative Cluster vs. Outcomes')


cancer_data.groupby('pr_status')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1, 1])
ax[1, 1].set_title('PR Status vs. Outcomes')


cancer_data.groupby('3-gene_classifier_subtype')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1,2])
ax[1, 2].set_title('3 Gene Class. Subtype vs. Outcomes')

#Remove legends in subplots:
for a in ax.flat:
    if a.get_legend():
        a.get_legend().remove()

# common legend outside the subplots
handles, labels = ax[1, 0].get_legend_handles_labels()
fig.legend(handles,  ['Died of Disease', 'Survived or Died of Other Causes'], loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2)


plt.subplots_adjust(wspace=0.5, hspace=.7)

plt.show();

### Observations of note:

Metaplastic breast cancer had the worst outcome

Neoplasm Histologic Grade shows a clear relationship with outcome; the lower the grade, the more patients who died of cancer. 
It looks like patients who did not die of cancer generally had fewer than 5 positive lymph nodes, and most had 0. Most of the patients with over 5 lymph nodes positive died of cancer. 

Negative ER status , negative PR status, and positive HER32 status were associated with better outcomes. 


## 2.5.4 Are there similar distributions in the data for the different cohorts? <a id='2.5.4_Cohorts'></a> 
I want to make sure that the cohorts have similar data distributions. I'm picking a few metrics that will give me an idea of the overall population. 

In [ ]:
#looking at the overall counts for the different cohorts
sns.countplot(data=cancer_data,x='cohort')
plt.title('Cohort Count')
plt.show();

In [ ]:
# plot of cohorts vs clinical attributes
fig, ax = plt.subplots(2,2, figsize=(12,8))
cancer_data.groupby('cohort')['death_from_cancer'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0, 0])
ax[0, 0].set_title('Cohort vs Survival')

cancer_data.groupby('cohort')['tumor_stage'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[0, 1])
ax[0, 1].set_title('Cohort vs. Tumor Stage')

#cancer_data.groupby('cohort')['tumor_size'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1,0])
#ax[1,0].set_title('Cohort vs Tumor Size')

cancer_data[cancer_data['cohort'] == 1]['age_at_diagnosis'].plot.hist(ax=ax[1, 0], alpha=0.5, stacked=True)
cancer_data[cancer_data['cohort'] == 2]['age_at_diagnosis'].plot.hist(ax=ax[1, 0], alpha=0.5, stacked=True)
cancer_data[cancer_data['cohort'] == 3]['age_at_diagnosis'].plot.hist(ax=ax[1, 0], alpha=0.5, stacked=True)
cancer_data[cancer_data['cohort'] == 4]['age_at_diagnosis'].plot.hist(ax=ax[1, 0], alpha=0.5, stacked=True)
ax[1, 0].set_title('Cohort vs Age at Diagnosis')
ax[1, 0].legend(['Cohort 1', 'Cohort 2', 'Cohort 3', 'Cohort 4'])

cancer_data[cancer_data['cohort'] == 1]['overall_survival_months'].plot.hist(ax=ax[1, 1], alpha=0.5, stacked=True)
cancer_data[cancer_data['cohort'] == 2]['overall_survival_months'].plot.hist(ax=ax[1, 1], alpha=0.5, stacked=True)
cancer_data[cancer_data['cohort'] == 3]['overall_survival_months'].plot.hist(ax=ax[1, 1], alpha=0.5, stacked=True)
cancer_data[cancer_data['cohort'] == 4]['overall_survival_months'].plot.hist(ax=ax[1, 1], alpha=0.5, stacked=True)
ax[1, 1].set_title('Cohort vs Overall Survival Months')
ax[1, 1].legend(['Cohort 1', 'Cohort 2', 'Cohort 3', 'Cohort 4'])

#plt.subplots_adjust(wspace=0.5, hspace=2.7)

plt.show();


In the above plots, cohort 3 seems to have a bit more death from disease, but they also have an overall older patient population, and above we saw that older patients are more likely to have poor outcomes. I also see that cohort 3 has the largest spread of survival months; however as this was the largest group it is not surprising. This is all consistent with what we've seen so far and I don't think we need to take into account the cohort, except with the missing stage data from cohort 4. 

## 2.5.5 Correlations Between Clinical Attributes   <a id='2.5.5_Attributes_Correlations'></a>

In [ ]:
# Segment out the clinical attribute data from cancer_data and look at data types. 
clinical = cancer_data.iloc[:,1:31]

clinical.dtypes

In [ ]:
#add back in patient_id column
clinical=clinical.join(cancer_data['patient_id'],how='left')

#put patient_id at the beginning of the DF, moving it from the end:
clinical.insert(0, 'patient_id', clinical.pop('patient_id'))

In [ ]:
# corr() cannot be done on objects, and some of our integers need to be converted to categories as they aren't continuous

clinical['chemotherapy']=clinical['chemotherapy'].astype('category')
clinical['cohort']=clinical['cohort'].astype('category')
clinical['radio_therapy']=clinical['radio_therapy'].astype('category')
clinical['neoplasm_histologic_grade']=clinical['neoplasm_histologic_grade'].astype('category')
clinical['overall_survival']=clinical['overall_survival'].astype('category')
clinical['tumor_stage']=clinical['tumor_stage'].astype('category')
clinical['hormone_therapy']=clinical['hormone_therapy'].astype('category')
#drp integrative_cluster as it is an object
clinical=clinical.drop(columns=['integrative_cluster'])

clinical.dtypes

So that we can do imputation and corr(), we need to do one hot encoding on the categories:

In [ ]:
# select the columns that are categories
clinical_cat = clinical.select_dtypes('category')
# make a list of the column labels
clinical_cat = clinical_cat.columns.values.tolist()
clinical_cat 


In [ ]:
# we also need the columns that are not categories
clinical_num = clinical.select_dtypes(['float64','int64'])
clinical_num.head()

In [ ]:
# One hot encoding to create separate columns for all the categories. Setting dummy_na=True so that I don't lose my missing data.
#clinical_dummies= pd.get_dummies(data=clinical,columns=clinical_cat,dummy_na=True)
clinical_dummies=pd.get_dummies(data=clinical, dummy_na=True)

clinical_dummies.head(10)

In [ ]:
print('shape clinical_dummies:',clinical_dummies.shape)

In [ ]:
# Find columns with all 0s
all_zero_cols = clinical_dummies.columns[(clinical_dummies == 0).all()]
print(all_zero_cols)

# Drop these columns
clinical_dummies.drop(columns=all_zero_cols, inplace=True)

In [ ]:
#list the column names
for col in clinical_dummies:
    print(col)

In [ ]:
# Confirm that we have no more missing data
clinical_dummies.isna().sum().sort_values(ascending=False)

I'm still missing 20 tumor_size entries. 

I'm going to see if tumor_size is correlated with tumor_stage, but I'm going to use my clinical df, before I did get_dummies. As tumor_stage is categorical, I will find the correlation with Spearman's rank correlation. 

In [ ]:
#First remove rows with missing tumor stage values
# Drop rows with missing values
clinical_dropped=clinical.dropna(subset=['tumor_size', 'tumor_stage'])


# Compute the Spearman's rank correlation coefficient between tumor_size and tumor_stage
corr, pval = spearmanr(clinical_dropped['tumor_size'], clinical_dropped['tumor_stage'])

# Print the correlation coefficient and p-value
print(f"Spearman's rank correlation coefficient: {corr:.2f}")
print(f"P-value: {pval:.4f}")

In [ ]:
# plot tumor size vs stage
plt.scatter(clinical_dropped['tumor_size'], clinical_dropped['tumor_stage'])
plt.xlabel('Tumor Size')
plt.ylabel('Tumor Stage')
plt.title('Tumor Size vs Stage')
plt.show()

There is a reasonably good correlation between tumor size and stage. As there is a small number (20) of missing values compared to the size of the dataset, I think it's better to impute this than to drop those columns. I was not able to deal with them with the get_dummies as it's not categorical. I'm using the median of the stages to impute as there are outliers. 

### 2.5.5a Impute the missing tumor_size with median of the associated stage  <a id='2.5.5a_impute_tumor_size'></a>

In [ ]:

# calculate the median tumor size for each tumor stage group
median_tumor_size = clinical.groupby('tumor_stage')['tumor_size'].median()
print(median_tumor_size)

In [ ]:
# find out if there are patients who have both of these columns missing data:

# count the number of rows where both tumor_size and tumor_stage are NaN
rows_with_both_nan = clinical[['tumor_size', 'tumor_stage']].isna().all(axis=1).sum()

print(rows_with_both_nan)

Unfortunately, most of the columns missing tumor_size are also missing tumor_stage.

In [ ]:
#Impute the 3 rows that have both of these columns: 

# loop over the rows with missing tumor size values
for index, row in clinical[clinical['tumor_size'].isnull()].iterrows():
    # get the tumor stage for the current row
    stage = row['tumor_stage']
    
    # if the median tumor size for the stage is available, replace the missing value with the median
    if stage in median_tumor_size:
        clinical.at[index, 'tumor_size'] = median_tumor_size[stage]


# Patients still missing size data:
print(clinical['tumor_size'].isnull().sum())

In [ ]:
# Now we'll just impute with the overall median for whether they died or not. 

outcome_size =clinical.groupby('death_from_cancer')['tumor_size'].median()
outcome_size
 

In [ ]:
#Replace missing size values with the median size for their outcome group.

for index, row in clinical[clinical['tumor_size'].isnull()].iterrows():
    outcome = row['death_from_cancer']
    if outcome in median_tumor_size:
        clinical.at[index, 'tumor_size'] = median_tumor_size[outcome]

# check the number of missing tumor_size values after imputation
print(clinical['tumor_size'].isnull().sum())

In [ ]:
# replace the tumor_size column in clinical_dummies:

clinical_dummies['tumor_size']=clinical['tumor_size']

clinical_dummies.isna().sum().sort_values(ascending=False)

Horray! No more missing data in our clinical attributes!!!

## 2.5.6 Correlation Matrix <a id='2.5.6_correlation_matrix'></a>

In [ ]:
# view correlation full matrix

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
corr_matrix = clinical_dummies.corr()
corr_matrix

This is a large matrix, and I'd like to print out the pairs that are significantly correlated.

In [ ]:

abs_corr = clinical_dummies.corr().abs()

unstacked = abs_corr.unstack()
corr_pairs = unstacked.sort_values(kind="quicksort", ascending=False)

print(corr_pairs[(corr_pairs < 1.0) & (corr_pairs > 0.5)])

None of the more strong correlations tell us anything about survival. 

For death_by_cancer, I see a weak correlation (0.24) with the number of lymph nodes positive and similarly weak correlation (0.27) with the nottingham prognostic index. I don't see any other correlations that could mean anything for death_by_cancer.



In [ ]:
#looking at box plot of the outcome vs lympth nodes. I see that people who survived overall had fewer lymph nodes positive. 

_ = sns.boxplot(x='death_from_cancer',y='lymph_nodes_examined_positive',data=clinical)
plt.show;

In [ ]:
# looking at outcome vs the nottingham prognostic index. Here I see that most people who died have an index over 4,
#and most who survived had an index under 4, even though the median for the two groups was similar.

#THIS SEEMS TO BE OUR BEST INDICATOR SO FAR OF OUTCOME PREDICTION

_ = sns.boxplot(x='death_from_cancer',y='nottingham_prognostic_index',data=clinical)
plt.show;

In [ ]:
#make a heat map of correlations

plt.figure(figsize=(40,20))

sns.heatmap(clinical_dummies.corr(), annot=False);

## 2.6 Z Score Data  <a id='2.6_z_score'></a>

### 2.6.1 How are the z scores connected to the genetic mutation data? <a id='2.6.1_z_score_mutation'></a>

How do the z scores relate to the mutation section of our data? Are there corresponding mutations for the z score columns?

In [ ]:
#extract z score columns:
z_scores=cancer_data.loc[:,'brca1':'ugt2b7']

#extract mutation columns:
mutation_cols = cancer_data.loc[:,'pik3ca_mut':]


In [ ]:
#I want to find which genes are both in z_scores and mutation_cols.

z_mut_match=[]
# Iterate over the z-score column names
for col in z_scores:
    gene_name = col  # Assuming the gene name is the same as the z-score column name
    mutations = f"{gene_name}_mut"
    if mutations in mutation_cols.columns:
        z_mut_match.append(col)
        
print(z_mut_match)

In [ ]:
print('number of columns in z_scores: ',z_scores.shape[1])
print('number of columns in mutation_cols: ',mutation_cols.shape[1])
print('number of matches between z scores and mutations: ',len(z_mut_match))
print('percent of z_scores with corresponding mutation: ',len(z_mut_match)/z_scores.shape[1]*100,'%')

print('number of z scores that do not have a match: ',489-len(z_mut_match))
print('number of mutations that do not have a match: ',173-len(z_mut_match))

### Conclusion:

Only about 34% of the z_score columns have a corresponding column for mutations, but all but 5 of the mutation columns have a matching z_score. However, we should still look at all of the z scores and all of the mutations because the fact that a gene is up-regulated or down-regulated may potentially have an impact on survival. I don't think I should try to tie the z scores and genes together in the data, I'll leave it as it is. 



## 2.6.2 Z Score Outcomes <a id='2.6.2_z_score_outcomes'></a>
Here I am investigating our z scores, and will see how they are related to patient outcomes.

### 2.6.2a Z Score Distributions  <a id='2.6.2a_z_score_dist'></a>

In [ ]:
#create dataframe of just z_scores
z_scores = cancer_data.iloc[:,32:520]
z_scores.head()

In [ ]:
#create one with patient IDs and outcome as well:

patient_id = cancer_data['patient_id']
outcome = cancer_data['death_from_cancer']
z_scores_id = pd.concat([patient_id, outcome, z_scores], axis=1)
z_scores_id.set_index('patient_id', inplace=True)

z_scores_id.head()

In [ ]:
z_scores.describe()

In [ ]:
z_scores.info()

In [ ]:
#find maximum z scores
max_z = z_scores.max().sort_values(ascending=False)
max_z.head(20)

In [ ]:
#minimum z scores

min_z = z_scores.min().sort_values(ascending=True)
min_z.head(20)

I see that our z scores go between about -7 and 20, but where do most of the z scores fall?

In [ ]:
# create a list of gene names to plot
genes_to_plot = z_scores.columns.to_list()

# create a list to hold the z scores for each gene
z_score_lists = []

# loop through each gene in genes_to_plot and add its z scores to the list
for gene in genes_to_plot:
    z_score_lists.append(z_scores[gene])

# plot the histograms for each gene on the same plot
plt.hist(z_score_lists, bins=20, label=genes_to_plot)

plt.xlabel('Z score')
plt.ylabel('Frequency')
plt.title('Histogram of Z scores for all genes')
plt.ylim(0,1100)

plt.show()

The above plot covers the range of all of the data, but the vast majority of our data is between -5 and 5. I want to zoom in here. 

In [ ]:
# Repeating the previous code block, but zoomed in to -5 to 5

for gene in genes_to_plot:
    z_score_lists.append(z_scores[gene])

plt.hist(z_score_lists, bins=20, label=genes_to_plot)

plt.xlabel('Z score')
plt.ylabel('Frequency')
plt.title('Histogram of Z scores for all genes')
plt.ylim(0,1100)
plt.xlim(-5,5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.show()

I see that the vast majority of our z scores fall between -3 and 4, and mostly beween -2 and 2. I would like to see if certain ranges of z scores are related to survival, as I suspect that the extreme z scores may be important.

In [ ]:
#Z SCORES OVER 4
# Create a boolean mask based on the condition z_scores over 4
mask = (z_scores_id.iloc[:, 2:] >= 4).any(axis=1)

# Apply the boolean mask to filter the rows
z_score_over_4 = z_scores_id.loc[mask]

#Z SCORES UNDER =4

mask2 = (z_scores_id.iloc[:, 2:] <= -4).any(axis=1)

# Apply the boolean mask to filter the rows
z_score_under_neg_4  = z_scores_id.loc[mask2]

z_score_under_neg_4.head()

In [ ]:
z_score_over_4.info()

In [ ]:
z_score_over_4.shape

In [ ]:
z_score_under_neg_4.info()

In [ ]:
z_score_under_neg_4.shape

In [ ]:
print('Number of patients with z scores over 4: ',z_score_over_4.shape[0])
print('Number of patients with z scores under -4: ',z_score_under_neg_4.shape[0])

It looks like 52% of our patients have at least 1 z score over 4, but only 5.5% have a z score under -4. Let's investigate this further to see how it may relate to outcome.

In [ ]:

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

# z scores under -4
sns.countplot(x='death_from_cancer', data=z_score_under_neg_4, ax=axes[0])
axes[0].set_xlabel('Outcome \n(0:Died of Disease, 1:Survived/Death from other causes)')
axes[0].set_ylabel('Count')
axes[0].set_title('Outcomes for Patients with Z Scores Under -4')

# full z scores
sns.countplot(x='death_from_cancer', data=z_scores_id, ax=axes[1])
axes[1].set_xlabel('Outcome \n(0:Died of Disease, 1:Survived/Death from other causes)')
axes[1].set_ylabel('Count')
axes[1].set_title('Outcomes for All Patients')

# z scores over 4
sns.countplot(x='death_from_cancer', data=z_score_over_4, ax=axes[2])
axes[2].set_xlabel('Outcome \n(0:Died of Disease, 1:Survived/Death from other causes)')
axes[2].set_ylabel('Count')
axes[2].set_title('Outcomes for Patients with Z Scores Over 4')

# Add spacing between subplots
fig.tight_layout()

# Show the plot
plt.show();


Below are printouts of the percents and counts for these three groups:

In [ ]:
# z scores under -4
survival_counts1 =z_score_under_neg_4['death_from_cancer'].value_counts()
print('Low Z Score: Percents Died vs Survived:\n',100*survival_counts1/len(z_score_under_neg_4))
print('\nLow Z Scores Counts:\n ',survival_counts1)

survival_counts2 = cancer_data['death_from_cancer'].value_counts()
print('\nAll Patients: Percents Died vs Survived:\n',100*survival_counts2/len(cancer_data))
print('\nAll Z Scores Counts:\n ',survival_counts2)

survival_counts3 =z_score_over_4['death_from_cancer'].value_counts()
print('\nHigh Z Score: Percents Died vs Survived:\n',100*survival_counts3/len(z_score_over_4))
print('\nAll Z Scores Counts:\n ',survival_counts3)

I can see tht 32.7% of all patients died from cancer, 36.5% for those with at least 1 z score under -4, and 35% of those with at least 1 z score over 4. 

However, our patient count is much lower for those with the low z scores. Is the change in proportion due to chance or is it significant? 

I will test our null hypothesis, which is that the z score makes no difference in patient outcome. As our outcome data is binary I should do a chi-squared test, but because our under -4 dataset only has 104 patients, I will also do a Fisher's Exact test. I'll do both to compare, but will rely more heavily on Fisher's Exact test. 

### 2.6.2b P-Values for Z Scores <a id='2.6.2a_p_values'></a>

Create Contingency Tables:

In [ ]:
# contingency table for z_score_over_4 data:

table_fisher_4 = pd.DataFrame({'Z Score over 4':[348,646 ], 'All Z Scores':[622,1281]},index=pd.Index(['Died of Disease', 'Did Not Die of Disease']))
table_fisher_4

In [ ]:
# contingency table for z_score_under_neg_4 data:

table_fisher_neg_4 = pd.DataFrame({'Z Score Under -4':[38,66 ], 'All Z Scores':[622,1281]},index=pd.Index(['Died of Disease', 'Did Not Die of Disease']))
table_fisher_neg_4

### P-Value for Z scores Under -4

In [ ]:
# Fisher's Exact Test (following this method: https://www.reneshbedre.com/blog/fisher-exact-test-python.html)
oddsr, p = fisher_exact(table=table_fisher_neg_4.to_numpy(), alternative='greater')
oddsr, p
print(f"Odds ratio: {oddsr:.2f}")
print(f"P-value: {p:.2e}")

In [ ]:
# Chi Squared Test, to see how it compares:

chi2, p, dof, expected = chi2_contingency(table_fisher_neg_4)
print('Chi-squared statistic:', chi2)
print('P-value:', p)

### P-Value for Z scores Over -4

In [ ]:
# Fisher's Exact Test (following this method: https://www.reneshbedre.com/blog/fisher-exact-test-python.html)
oddsr, p = fisher_exact(table=table_fisher_4.to_numpy(), alternative='greater')
oddsr, p
print(f"Odds ratio: {oddsr:.2f}")
print(f"P-value: {p:.2e}")

In [ ]:
# Chi Squared Test, to see how it compares:

chi2, p, dof, expected = chi2_contingency(table_fisher_4)
print('Chi-squared statistic:', chi2)
print('P-value:', p)

### Observations:
The p values for extreme z scores are not small enough to reject the null hypothesis. So we cannot say that z scores impact survival outcomes.  


## 2.6.3 Correlation of Z Scores and Outcome <a id='2.6.3_corr'></a>

In [ ]:
z_score_over_4.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
correlation=[]
for col in z_score_over_4.columns:
    corr = z_score_over_4[col].corr(z_score_over_4['death_from_cancer'])
    correlation.append(corr)

  
correlation.pop(0)
ax.hist(correlation,  bins=30)
ax.set_xlabel("Correlation")
ax.set_ylabel("Count")
ax.set_title('Correlation of Z Score over 4 and Outcome')

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
correlation=[]
for col in z_score_under_neg_4.columns:
    corr = z_score_under_neg_4[col].corr(z_score_under_neg_4['death_from_cancer'])
    correlation.append(corr)
    
  
correlation.pop(0)
ax.hist(correlation,  bins=30)
ax.set_xlabel("Correlation")
ax.set_ylabel("Count")
ax.set_title('Correlation of Z Score Under -4 and Outcome')

plt.show()


Now we are getting somewhere! The positive z scores don't show any correlation, but there is more correlation with the patients with z_scores under -4. 

Next I want to find what columns contain these low values, and find the p value. 

In [ ]:
# Filter the DataFrame to keep only the rows where the values are <= -4

#rename df to be easier to type:
low_z = z_score_under_neg_4

#looking to see which genes have low z scores (under -4)
low_z_bool = z_scores_id[(z_scores_id.iloc[:, 2:] < -4)].any().sort_values(ascending=False)
low_z_bool.head()

In [ ]:
#filtering my dataframe to make it so all values over -4 are NAN

# Create a boolean mask of values less than or equal to -4
mask = low_z.iloc[:, 2:] <= -4

# Use the where method to replace values greater than -4 with NaN
low_z_filtered = low_z.iloc[:, 2:].where(mask, other=np.nan)

# Combine the patient_id and death_from_cancer columns with the filtered z-score data
low_z_filtered = pd.concat([low_z.iloc[:, :1], low_z_filtered], axis=1)
low_z_filtered.head()


In [ ]:
low_z_filtered.info()

In [ ]:
from scipy.stats import mannwhitneyu
# null hypothsis is that z score makes no difference on outcome.Do a Mann Whitney U Test.
# this test is used when data are not normally distributed. Tests if sample mans ar equal or not, often used in medical trials. 

# define the two groups based on the death_from_cancer column
survived_other_causes= low_z[low_z['death_from_cancer'] == 1]
death_disease = low_z[low_z['death_from_cancer'] == 0]

# loop over each column and perform the Mann-Whitney U-test
p_values = []
for col in low_z.columns[2:]:
    stat, p = mannwhitneyu(survived_other_causes[col].dropna(), death_disease[col].dropna())
    p_values.append(p)

# create a new dataframe with the p-values for each column
p_values_df = pd.DataFrame({'gene': low_z.columns[2:], 'p_value': p_values})
p_values_df.head()

In [ ]:
p_significant = p_values_df[p_values_df['p_value']<= 0.05]
p_significant.info()

In [ ]:
p_significant.sort_values(by='p_value')

Now I have the above genes where my null hypothesis is false, so these genes may impact outcome if the z score is under -4. I don't know what to do about this now. I have 127 genes of interest out of the original 487. 

In [ ]:
#list of columns with p score > 0.05
sig_genes = p_significant['gene'].tolist()
print(sig_genes)

In [ ]:
# looking at the correlation coefficant and p value for the gene with the most significant p value:
corr, p = pearsonr(z_score_under_neg_4['cyb5a'], z_score_under_neg_4['death_from_cancer'])

# Print the correlation coefficient and its associated p-value
print('Correlation coefficient:', corr)
print('P-value:', p)


Now I want to create a new dataframe with just these genes, and see what I see there. 

In [ ]:
# First make a new dataframe with the patient id and outcome:

z_scores_sig = cancer_data[['patient_id','death_from_cancer']]

# Now add in the gene columns if they are in sig_genes

# select the columns I want from 'cancer_data'
cancer_data_subset = cancer_data.loc[:, sig_genes]

# merge the subset dataframe with 'z_scores_sig' based on their indices
z_scores_sig = z_scores_sig.merge(cancer_data_subset, left_index=True, right_index=True)

z_scores_sig.head()

In [ ]:
z_scores_sig.info()

CONCLUSION:There are 129 genes have are weakly correlated to survival. I'm leaving everything as-is for now. 

## 2.7 Genetic Mutation Data <a id='2.6.7_genetic'></a>

How many columns of genetic mutations are in the dataset?

In [ ]:
#getting a list of gene mutation columns
gene_mut_columns = [col for col in cancer_data.columns if '_mut' in col]
number_gen_mut_columns = len(gene_mut_columns)
print('number of gene mutation columns: ',number_gen_mut_columns)

I have seen previously that some of the columns have two gene mutation entries for one patient. How often does this show up?

I have seen previously that some of the columns have two gene mutation entries for one patient. How often does this show up?

In [ ]:
#first make a dataframe that's a subset of cancer_data, containing only the genetic mutation data. 
gene_mut = cancer_data.loc[:,'pik3ca_mut':]
gene_mut.head()


I know that in the data, multiple mutations for a single patient and gene are entered by including a space between each mutation. I will search for spaces, but first need to strip spaces from the beginning and end of each entry.

In [ ]:
#convert into object
gene_mut = gene_mut.astype('object')

#strip the spaces from either side of entries in dataframe

gene_mut_stripped = gene_mut.apply(lambda x: x.str.strip())


Now I will get the count of the genetic mutations that have more than one mutation listed at a time:

In [ ]:

mutation_counts = gene_mut.apply(lambda x: x.str.count(' ').sum())
non_zero_counts = mutation_counts[mutation_counts > 0]
total_non_zero_counts = non_zero_counts.sum()
print('TOTAL number of occurances of more than one mutation per gene per patient: ',total_non_zero_counts)

#and also want to know the total number of mutations listed overall:
#finding the entries that are non zero
non_zero_mutation = gene_mut.ne('0')
#summing the boolean values and then converting to scalar with second sum. 
#This does not include the 'extras' where there are more than one mutation listed at a time 
total_mut_count = non_zero_mutation.sum().sum()
print('total mutation count: ',total_mut_count)

print('\ntally of mutiple mutations listed per patient, per gene:\n',non_zero_counts.to_string())

I now want to know if any patients have more than 2 gene mutations listed at a time. 

In [ ]:
#define a funtion to search for more than 2 mutation in a cell:

def count_mut(s):
    return s.count(' ') + 1

#apply the function:

muts_3plus = gene_mut_stripped.apply(lambda x: x.apply(count_mut))

# find the columns that contain at least one cell with 3 mutations
columns_muts_3 = muts_3plus.columns[muts_3plus.apply(lambda x: (x == 3).any())]

print('columns with 3 mutations per patient:\n ',columns_muts_3)

#see if any columns contain 4 or more mutations

columns_muts_4 = muts_3plus.columns[muts_3plus.apply(lambda x: (x == 4).any())]

print('\ncolumns with 4 mutations per patient: \n ',columns_muts_4)


columns_muts_5 = muts_3plus.columns[muts_3plus.apply(lambda x: (x >= 5).any())]

print('\ncolumns with 5 mutations per patient: \n ',columns_muts_5)


There's no easy end in sight here, so I'm going to do a count to find the maximum number of mutations. 

In [ ]:
# Define a function to count the number of mutations in a cell
def count_mutations(cell):
    #check if cell contents is a string
    if isinstance(cell, str):
    #if so, splits it into a list and counts how many mutations are in the list
        return len(cell.split())
    else:
        return 0

# Apply the function to each cell in the dataframe
mutation_counts_cell = gene_mut_stripped.applymap(count_mutations)

# Find the maximum number of mutations
max_mutations = mutation_counts_cell.max().max()

print(f"The maximum number of mutations in a single cell is {max_mutations}.")


<b>Summary so far:</b> We have 173 columns of genetic mutation data, with 9728 total cells containing mutations, and 851 of these contain more than 1 mutation. The maximum number of mutations in a single cell is 10, but the majority with multiple mutations per cell only have 2. 

Next steps: I need to figure out how to split the data, and then I can do more EDA once that happens. 

In [ ]:
#First I want to add the patient_id column back to the mutation data in case I need this. 
patient_id = cancer_data['patient_id']
gene_mut_merged = pd.concat([patient_id, gene_mut_stripped], axis=1)
gene_mut_merged.set_index('patient_id')

### 2.7.1 Convert gene mutation columns into binary  <a id='2.7.1_binary'></a>
Our dataset is getting large and complicated, and I think that the first thing we want to do is to convert each cell into a 0 or 1 to indicate if there are mutation(s) present or not.
    

In [ ]:
mutations = gene_mut_merged.copy()

for i in range(len(mutations)):
    for j in range(1, len(mutations.columns)):
        if len(str(mutations.iloc[i,j])) > 1:
            mutations.iloc[i,j] = 1
        else:
            mutations.iloc[i,j] = 0

    

In [ ]:
mutations.head()

In [ ]:
#I want to look at just the mutations without the patient_id.

mutations_no_id = mutations.iloc[:,1:]

mutations_no_id.info()


I see that the dtype is still object, so I should change it to int.

In [ ]:
mutations = mutations.astype('int')
mutations_no_id = mutations_no_id.astype('int')
mutations.info()

In [ ]:
mutations_no_id.describe()

### 2.7.2 Correlations of Mutation Data <a id='2.7.2_corr'></a>

In [ ]:
# Find the most common 25 genes for mutations to take place, and count them.
mut_sum = mutations_no_id.sum()
top_25 = mut_sum.head(25)
top_25

So I see the most common mutations that our patients have. I would like to look at these on a plot. 

In [ ]:
sns.barplot(x=top_25.index, y = top_25.values)
plt.title("Top 25 Genes with Mutations")
plt.xlabel("Gene")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show;

In [ ]:
#insert the overall_survival column into 'mutation' df - 

mutations.insert(1, "overall_survival", cancer_data["overall_survival"])
mutations.head()

In [ ]:
# plot histogram of Pearson Correlation Coefficient v number of genes with that correlation
# Doing this a similar way as I found here: https://www.kaggle.com/code/raghadalharbi/breast-cancer-survival-prediction-acc-0-779 

fig, ax = plt.subplots(figsize=(10,4))
correlation=[]
for col in mutations.drop(['patient_id'], axis = 1).columns:
    corr = mutations[[col,'overall_survival']].corr()['overall_survival'][col]
    correlation.append(corr)
    
correlation.pop(0)
ax.hist(correlation,  bins=30)
ax.set_xlabel("Correlation")
ax.set_ylabel("Frequency of a Correlation")
ax.set_title('Number of Genes with a Given Correlation with Overall Survival')


plt.show();

In the above plot, we see that there are more genes correlated with patient death than with survival, but these do not show enough correlation to further investigate down this path. 

# 2.8 Preparation for export   <a id='2.8_prep'></a>

I need to tie our data back together and confirm that everything looks good so that we can move onto the next step

The dataframes we want to look at are:
* clinical_dummies
* zscores_id
* mutations

In [ ]:
# we previously saved the patient_id as a series, checking it out here. 
patient_id.info()

In [ ]:
# Looking at our clinical dataframe that was converted to numeric already with get_dummies method, inspecting this here.
clinical_dummies.head()

In [ ]:
# Reset index of our clinical and mutations to 'patient_id' and make naming convention consistent

clinical_id = clinical_dummies.set_index('patient_id')
mutations_id = mutations.set_index('patient_id')
clinical_id.head()

In [ ]:
# Drop the death_from_cancer column from z_scores_id and mutations_id, so that when we join our 
#dataframes together we don't have repeats
z_scores_no_outcome = z_scores_id.drop('death_from_cancer',axis=1)
mutations_no_outcome = mutations_id.drop('overall_survival',axis=1)

In [ ]:
z_scores_no_outcome.head()

In [ ]:
mutations_no_outcome.head

Merge our datasets back together

In [ ]:
num_data = pd.merge(clinical_id,z_scores_no_outcome, left_index=True, right_index=True)
num_data = pd.merge(num_data, mutations_no_outcome, left_index=True, right_index=True)
num_data.head()

In [ ]:
# Confirm that there are no NANs left:
num_data.isnull().values.any()

In [ ]:
# Compare current shape to original shape
print('current shape: ',num_data.shape)
print('original shape: ',cancer_data.shape)

This is what I expect - I didn't drop any patients, and I did add extra columns when I expanded the clinical data.

In [ ]:
# check datatypes
num_data.dtypes


I see a lot of uint8 in my clinical data - look into this.

In [ ]:
num_data.info()

In [ ]:
# convert uint8 to int64

for col in num_data.columns:
    if num_data[col].dtype == 'uint8':
        num_data[col] = num_data[col].astype('int64')
        
 
# convert int32 to int64: 

for col in num_data.columns:
    if num_data[col].dtype == 'int32':
        num_data[col] = num_data[col].astype('int64')
num_data.dtypes

In [ ]:
num_data.head()

<b>STATUS:</b> I have a dataframe 'num_data', which contains no missing data and everything is numeric. I am now ready for the preprocessing and modeling section of this project. 

## 2.9 Exporting the Data  <a id='2.9_export'></a>

In [ ]:
# export to a parquet so we can save data type
num_data.to_parquet(r'C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\capstone 2\num_data.parquet', index=False)

## 2.10 Summary   <a id='2.10_summary'></a>
CLINICAL DATA: The average (and median) age of patients is around 61, and the vast majority had invasive ductal carcinoma. Most patients did not get chemo, but did get radiotherapy and hormone therapy. Frequency of mutation counts peaked at 7, there is a very large spread of positive lymph nodes (though most patients have under 10 or none), and there is also a large spread of tumor size, though again most are under 10 cm. 
I decided to change our key metric of ‘death_by_cancer’ from categorical to binary, where 0 indicates death from disease and for 1 I combined survived and death from other causes. This is because we are interested in whether or not patients died from breast cancer, and other causes of death can skew the results. 
Comparing treatment modalities for outcomes overall doesn’t reveal anything obvious; outcomes are roughly the same for hormone therapy and radiotherapy, and outcomes are slightly worse for patients who had breast conserving surgery and no chemotherapy, which makes sense. 
Tumor stage appears to be the best indicator of outcome, as the proportion of patients who die from disease increases fairly evenly with stage 0-4. Unfortunately, we are missing 26% of this data because one of the cohorts did not record this metric.  As there is a reasonably good connection between tumor size and stage (with some significant outliers) I decided to impute the missing tumor stage data from the mean tumor size for each stage for the patients that had this recorded. Unfortunately, most did not, so I imputed with the median tumor size for the patient’s outcome for those remaining missing values. 

When looking at our clinical attributes vs outcomes, metaplastic breast cancer had the worst outcome. Neoplasm Histologic Grade shows a clear relationship with outcome; the lower the grade, the more patients who died of cancer. It looks like patients who did not die of cancer generally had fewer than 5 positive lymph nodes, and most had 0. Most of the patients with over 5 lymph nodes positive died of cancer. Negative ER status , negative PR status, and positive HER32 status were associated with better outcomes.
I looked at the correlations between clinical attributes, and did not see any strong correlations with outcomes. I did note that there is a weak correlation between the number of positive lymph nodes and outcome, and made a box plot of this. I also noted that the Nottingham Prognostic Index showed a weak correlation, and when I looked at a box plot of this index vs outcomes, I saw that most people wo died of disease have an index over 4, and most patients who survived had an index under 4, even though the median for the two groups was very similar. Overall this seems to be a decent indicator of outcome. 

Z SCORES: I found that the z scores range from -7 to 20, with the vast majority falling between -3 and 4, and mostly between -2 and 2. I suspected that the z score outliers may impacted survival, and I looked into whether or not outcomes differed for patients with at least one z score over 4 (52% of patients) or under -4 (5.5% of patients). There was a difference in the outcomes, but when I did a null hypothesis test the p values were not small enough to say that the differences in outcomes is significant.

When looking at the correlation of z scores over 4/under -4 and outcome, I saw no correlation at all with z scores over 4, but some weak correlations with z scores under -4. I should also note that the p value for the low z scores was lower (0.1), so it makes sense that we may see a bit more correlation here. I looked to see which genes have significant correlations, and I found 127 genes with a p value under 0.05, indicating that these are significantly (though weakly) correlated with survival. 

GENETIC MUTATIONS: I found that the most mutations happened on the pika3ca, tp53, muc15, ahnak2 and kmt2c genes, and looked at the patients that had multiple mutations on a single gene. We have 173 columns of genetic mutation data, with 9728 total cells containing mutations, and 851 of these contain more than 1 mutation. The maximum number of mutations in a single cell is 10, but the majority with multiple mutations per cell only have 2. To avoid having an enormous data set before it’s necessary, I decided to convert this section of the data into binary ‘mutations’ and ‘no mutations’, indicated by 1 or 0. If I find something significant here, I can go back and look and see if particular mutations make an important difference. 
I also looked to see if there was a correlation between outcomes and frequency of the presence of mutations on certain genes, and did not observe anything of note. 

I merged our 3 sections of the data back together, and confirmed that there is no missing data and everything is numeric and either a float

The next notebook is titled 'Capstone2_modeling'.